In [7]:
import duckdb

In [8]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [ ]:
# Verificando os dados na tabela bronze_z0019
df = con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head(10)

,NATBR,MARTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-01-19 20:27:43.188722
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2026-01-19 20:27:43.188722
2,10003,PREGO,BT10,100,50,z0019_1.csv,2026-01-19 20:27:43.188722
3,10004,SERRA,BT50,100,200,z0019_2.csv,2026-01-19 20:34:25.981050
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2026-01-19 20:34:25.981050
5,10003,PREGO,BT10,100,50,z0019_2.csv,2026-01-19 20:34:25.981050


In [22]:
df = con.execute("""
            SELECT *
            FROM(
                SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                FROM bronze_z0019
                WHERE data_ingestao >= '2026-01-19'
            ) WHERE row = 1
    
""").fetch_df()

df.head(10)

,NATBR,MARTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10004,SERRA,BT50,100,200,z0019_2.csv,2026-01-19 20:34:25.981050,1
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2026-01-19 20:27:43.188722,1
2,10005,MACHADO,BT50,100,100,z0019_2.csv,2026-01-19 20:34:25.981050,1
3,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-01-19 20:27:43.188722,1
4,10003,PREGO,BT10,100,50,z0019_2.csv,2026-01-19 20:34:25.981050,1


In [23]:
# Vamos trabalhar agora na parte de modificações 
# Deletar as colunas nome_arquivo e data_ingestao

df_final = df.drop(columns=['nome_arquivo', 'data_ingestao','row'])
df_final

,NATBR,MARTX,WERKS,MAINS,LABST
0,10004,SERRA,BT50,100,200
1,10002,MARTELO,BT50,100,1500
2,10005,MACHADO,BT50,100,100
3,10001,PARAFUSO,BT10,100,100
4,10003,PREGO,BT10,100,50


# Glossário dos nomes da tabela

- NATBR = id
- MAKTX = nome
- WERKS	= categoria
- MAINS	= fornecedor
- LABST = preco

In [24]:
# Renomeando as colunas de acordo com o glossário
df_final = df_final.rename(columns={"NATBR":"id"})
df_final = df_final.rename(columns={"MARTX":"nome_produto"})
df_final = df_final.rename(columns={"WERKS":"id_categoria"})
df_final = df_final.rename(columns={"MAINS":"id_fornecedor"})
df_final = df_final.rename(columns={"LABST":"vl_preco"})

df_final.head(10)

,id,nome_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200
1,10002,MARTELO,BT50,100,1500
2,10005,MACHADO,BT50,100,100
3,10001,PARAFUSO,BT10,100,100
4,10003,PREGO,BT10,100,50


In [25]:
# Verificando os tipos de informação
df_final.dtypes

id               object
nome_produto     object
id_categoria     object
id_fornecedor    object
vl_preco         object
dtype: object

In [26]:
# Criando um cópia do df_final
df2 = df_final


In [32]:
# modificando os tipos de dados
df2 = df2.astype(
    {
        'id': int,
        'nome_produto': str,
        'id_categoria': str,
        'id_fornecedor': int,
        'vl_preco': float
    }
)

df2.head(10)
df2.dtypes

id                 int32
nome_produto      object
id_categoria      object
id_fornecedor      int32
vl_preco         float64
dtype: object

In [ ]:
con.execute("""
    CREATE TABLE IF NOT EXISTS produtos(
        id BIGINT,
        nome_produto TEXT,
        id_categoria TEXT,
        id_fornecedor BIGINT,
        vl_preco FLOAT
    )
""")

In [ ]:
df2.head(10)


,id,nome_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200.0
1,10002,MARTELO,BT50,100,1500.0
2,10005,MACHADO,BT50,100,100.0
3,10001,PARAFUSO,BT10,100,100.0
4,10003,PREGO,BT10,100,50.0


In [35]:
# Fazendo a ingestão dos dados na tabela produtos
con.execute("""
    INSERT INTO produtos
    SELECT * FROM df2
""")

In [36]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

,id,nome_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200.0
1,10002,MARTELO,BT50,100,1500.0
2,10005,MACHADO,BT50,100,100.0
3,10001,PARAFUSO,BT10,100,100.0
4,10003,PREGO,BT10,100,50.0


In [37]:
con.close()